In [1]:
# -*- coding: utf-8 -*-

import pandas as pd
data = pd.read_csv('./data/1.2/tianchi_fresh_comp_train_user.csv', sep = ',',encoding = 'utf-8')

In [2]:
data

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,10001082,285259775,1,97lk14c,4076,2014-12-08 18
1,10001082,4368907,1,NaN,5503,2014-12-12 12
2,10001082,4368907,1,NaN,5503,2014-12-12 12
3,10001082,53616768,1,NaN,9762,2014-12-02 15
4,10001082,151466952,1,NaN,5232,2014-12-12 11
...,...,...,...,...,...,...
23291022,65341491,259008790,1,NaN,13164,2014-12-03 12
23291023,65341491,336404938,1,NaN,13164,2014-12-03 12
23291024,65341491,52142024,1,95qhbsu,5201,2014-12-10 22
23291025,65341491,250557965,1,NaN,13164,2014-12-03 12


# （1）查看数据表

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23291027 entries, 0 to 23291026
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        int64 
 1   item_id        int64 
 2   behavior_type  int64 
 3   user_geohash   object
 4   item_category  int64 
 5   time           object
dtypes: int64(4), object(2)
memory usage: 1.0+ GB


# （2）去除重复值

In [4]:
data.drop_duplicates(inplace = True)   #inplace = True表示直接在data中修改
len(data)

15463110

# （3）查看缺失值

In [5]:
data.isnull().sum()

user_id                0
item_id                0
behavior_type          0
user_geohash     8207386
item_category          0
time                   0
dtype: int64

# （4）变换数据表

In [6]:
df = data['time'].str.split(' ', expand=True) 
df

,0,1
0,2014-12-08,18
1,2014-12-12,12
3,2014-12-02,15
4,2014-12-12,11
5,2014-12-02,15
...,...,...
23291015,2014-12-18,01
23291018,2014-12-08,12
23291019,2014-12-08,12
23291023,2014-12-03,12


In [7]:
df = data['time'].str.split(' ') 
df

0           [2014-12-08, 18]
1           [2014-12-12, 12]
3           [2014-12-02, 15]
4           [2014-12-12, 11]
5           [2014-12-02, 15]
                  ...       
23291015    [2014-12-18, 01]
23291018    [2014-12-08, 12]
23291019    [2014-12-08, 12]
23291023    [2014-12-03, 12]
23291024    [2014-12-10, 22]
Name: time, Length: 15463110, dtype: object

### 将“time”字段拆分为日期字段“date”和小时字段“hour”

In [8]:
df = data['time'].str.split(' ', expand=True)  #将原表中'time'的日期和时间（小时）拆开
data['date'] = df.iloc[:, 0]
data['time'] = df.iloc[:, 1]
data.rename(columns={'time':'hour'},inplace=True)   # 将列名'time'改为'hour'

In [9]:
data.head()

,user_id,item_id,behavior_type,user_geohash,item_category,hour,date
0,10001082,285259775,1,97lk14c,4076,18,2014-12-08
1,10001082,4368907,1,NaN,5503,12,2014-12-12
3,10001082,53616768,1,NaN,9762,15,2014-12-02
4,10001082,151466952,1,NaN,5232,11,2014-12-12
5,10001082,53616768,4,NaN,9762,15,2014-12-02


### 将“behavior_type”字段拆分为“浏览”“收藏”“加购物车”“购买”等四个“0-1”型字段

In [10]:
data['收藏'] = 0
data['加购物车'] = 0
data['购买'] = 0

data.loc[data['behavior_type'] == 2, '收藏'] = 1
data.loc[data['behavior_type'] == 3, '加购物车'] = 1
data.loc[data['behavior_type'] == 4, '购买'] = 1
data.loc[data['behavior_type'] != 1, 'behavior_type'] = 0
data.rename(columns={'behavior_type':'浏览'},inplace=True)   # 将列名'behavior_type'改为'浏览'
data.rename(columns={'user_id':'用户'},inplace=True)   # 将列名'user_id'改为'用户'
data.rename(columns={'item_id':'商品'},inplace=True)   # 将列名'item_id'改为'商品'
data.rename(columns={'item_category':'品类'},inplace=True)   # 将列名'item_category'改为'品类'
data.rename(columns={'user_geohash':'地理位置'},inplace=True)   # 将列名'user_geohash'改为'地理位置'

In [11]:
data = data[['用户','商品','品类','地理位置','date','hour','浏览','收藏','加购物车','购买']]

In [12]:
data.head()

,用户,商品,品类,地理位置,date,hour,浏览,收藏,加购物车,购买
0,10001082,285259775,4076,97lk14c,2014-12-08,18,1,0,0,0
1,10001082,4368907,5503,NaN,2014-12-12,12,1,0,0,0
3,10001082,53616768,9762,NaN,2014-12-02,15,1,0,0,0
4,10001082,151466952,5232,NaN,2014-12-12,11,1,0,0,0
5,10001082,53616768,9762,NaN,2014-12-02,15,0,0,0,1


In [13]:
data.to_csv('./data/1.2/behavior.csv',encoding = 'utf-8',index=False)

### 生成新数据文件——用户表users.csv

In [14]:
# -*- coding: utf-8 -*-

import pandas as pd
data = pd.read_csv('./data/1.2/behavior.csv', sep = ',',encoding = 'utf-8')

In [15]:
data1 = data['用户']
data1.drop_duplicates(inplace = True)
data1

0            10001082
122         100029775
379         100068031
751         100076491
1158        100087810
              ...    
15458935     65186019
15461365     65200062
15462046     65222739
15462552     65328951
15462824     65341491
Name: 用户, Length: 20000, dtype: int64

In [16]:
data1.to_csv('./data/1.2/user.csv',encoding = 'utf-8',index=False, header = True)

In [17]:
data1

0            10001082
122         100029775
379         100068031
751         100076491
1158        100087810
              ...    
15458935     65186019
15461365     65200062
15462046     65222739
15462552     65328951
15462824     65341491
Name: 用户, Length: 20000, dtype: int64